In [ ]:
!nvidia-smi

Mon Oct 17 00:26:58 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!unzip -qq "./drive/MyDrive/data/open.zip"

In [ ]:
import glob
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import argparse
import random
import os
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import torchvision.models as models

from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--epochs', type=int, default=20)
parser.add_argument('--lr', type=float, default=0.001)
parser.add_argument('--batch_size', type=int, default=32)
parser.add_argument('--seed', type=int, default=999)
parser.add_argument('--img_size', type=int, default=224)
parser.add_argument('--device', default='cuda')

args = parser.parse_args('')
args

Namespace(batch_size=32, device='cuda', epochs=20, img_size=224, lr=0.001, seed=999)

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(args.seed)

In [ ]:
df = pd.read_csv('./train.csv')
# 잘못된 label 수정
df['artist'][3896] = 'Titian'
df['id'][3986] = 3986
df['artist'][3986] = 'Alfred Sisley'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

In [ ]:
le = preprocessing.LabelEncoder()
df['artist'] = le.fit_transform(df['artist'].values)
df.head()

,id,img_path,artist
0,0,./train/0000.jpg,9
1,1,./train/0001.jpg,48
2,2,./train/0002.jpg,7
3,3,./train/0003.jpg,10
4,4,./train/0004.jpg,24


In [ ]:
train_df, val_df, _, _ = train_test_split(df, df['artist'].values, test_size=0.2, random_state=args.seed)

In [ ]:
def get_data(df, infer=False):
    if infer:
        return df['img_path'].values
    return df['img_path'].values, df['artist'].values

In [ ]:
train_img_paths, train_labels = get_data(train_df)
val_img_paths, val_labels = get_data(val_df)

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, img_paths, labels, transforms=None):
        self.img_paths = img_paths
        self.labels = labels
        self.transforms = transforms

    def __getitem__(self, index):
        img_path = self.img_paths[index]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # 이미지 길이 1 / 2

        if self.labels is not None: # train
          h, w, _ = image.shape
          size = (h, w)
          inner_transform = A.Compose([A.RandomCrop(width=w//2, height=h//2)])
        else: # test
          h, w, _ = image.shape
          size = (2*h, 2*w)
        
        if self.transforms is not None:
            if self.labels is not None: # train
              image = inner_transform(image=image)['image']
            image = self.transforms(image=image)['image']
        
        if self.labels is not None:
            label = self.labels[index]
            return image, label # size
        else:
            return image # size
    
    def __len__(self):
        return len(self.img_paths)

In [ ]:
# random flip
train_transform = A.Compose([
                            A.HorizontalFlip(),
                            A.Resize(args.img_size,args.img_size),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(args.img_size,args.img_size),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [ ]:
train_dataset = CustomDataset(train_img_paths, train_labels, train_transform)
train_loader = DataLoader(train_dataset, batch_size = args.batch_size, shuffle=True, num_workers=0)

val_dataset = CustomDataset(val_img_paths, val_labels, test_transform)
val_loader = DataLoader(val_dataset, batch_size=args.batch_size, shuffle=False, num_workers=0)

In [ ]:
test = models.efficientnet_b4()
test.classifier

Sequential(
  (0): Dropout(p=0.4, inplace=True)
  (1): Linear(in_features=1792, out_features=1000, bias=True)
)

In [ ]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(BaseModel, self).__init__()
        self.backbone = models.efficientnet_b4(weights='DEFAULT')
        self.backbone.classifier = nn.Sequential()
        self.drop = nn.Dropout(0.3, inplace=True)
        self.clf = nn.Sequential(
            nn.Linear(1792,  512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2, inplace=False),
            nn.Dropout(p=0.4, inplace=True),
            nn.Linear(512, num_classes)
        )
        
    def forward(self, x):
        x = self.backbone(x)

        # h = size[0]
        # w = size[1]

        # why not working?
        # h = h.view(h.size(), 1)
        # w = w.view(w.size(), 1)
        # h = h.view(h.size(0), 1)
        # w = w.view(w.size(0), 1)
        x = self.drop(x)
        # x = torch.cat((x, h, w), 1) # 여기 cat부분이 cnn부분 끝나고 분류하는 layer들어가기 전에 붙인 부분입니다.
        x = self.clf(x)
        return x

In [ ]:
def competition_metric(true, pred):
    return f1_score(true, pred, average="macro")

def validation(model, criterion, test_loader, device):
    model.eval()
    
    model_preds = []
    true_labels = []
    
    val_loss = []
    
    with torch.no_grad():
        for img, label in tqdm(test_loader):
            img, label = img.float().to(device), label.to(device)
            # h = size[0].to(device)
            # w = size[1].to(device)
            # model_pred = model(img, (h, w))
            model_pred = model(img)
            
            loss = criterion(model_pred, label)
            
            val_loss.append(loss.item())
            
            model_preds += model_pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += label.detach().cpu().numpy().tolist()
        
    val_f1 = competition_metric(true_labels, model_preds)
    return np.mean(val_loss), val_f1

In [ ]:
def train(model, optimizer, train_loader, test_loader, scheduler, device):
    model.to(device)

    criterion = nn.CrossEntropyLoss().to(device)
    
    best_score = 0
    best_model = None
    
    for epoch in range(1, args.epochs+1):
        model.train()
        train_loss = []
        for img, label in tqdm(train_loader):
            img, label = img.float().to(device), label.to(device)
            # h = size[0].to(device)
            # w = size[1].to(device)
            optimizer.zero_grad()

            # model_pred = model(img, (h, w))
            model_pred = model(img)

            
            loss = criterion(model_pred, label)

            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())

        tr_loss = np.mean(train_loss)
            
        val_loss, val_score = validation(model, criterion, test_loader, device)
            
        print(f'Epoch [{epoch}], Train Loss : [{tr_loss:.5f}] Val Loss : [{val_loss:.5f}] Val F1 Score : [{val_score:.5f}]')
        
        if scheduler is not None:
            scheduler.step()
            
        if best_score < val_score:
            best_model = model
            best_score = val_score
        
    return best_model

In [ ]:
# ckpt = torch.load('/content/drive/MyDrive/data/model/')
# model = BaseModel().to(args.device)
model = BaseModel()
# model.load_state_dict(ckpt['model'])
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = args.lr)
# optimizer.load_state_dict(ckpt['optim'])
scheduler = None
# scheduler = torch.optim.StepLR(optimizer, step_size=15, gamma=0.1)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, args.device)

100%|██████████| 37/37 [00:22<00:00,  1.64it/s]


Epoch [1], Train Loss : [2.34704] Val Loss : [1.51729] Val F1 Score : [0.44103]


100%|██████████| 37/37 [00:21<00:00,  1.76it/s]


Epoch [2], Train Loss : [1.38245] Val Loss : [1.16689] Val F1 Score : [0.57445]


100%|██████████| 37/37 [00:20<00:00,  1.77it/s]


...

Epoch [18], Train Loss : [0.23094] Val Loss : [0.89646] Val F1 Score : [0.74289]


100%|██████████| 37/37 [00:21<00:00,  1.75it/s]


Epoch [19], Train Loss : [0.24842] Val Loss : [0.89431] Val F1 Score : [0.74034]


100%|██████████| 37/37 [00:21<00:00,  1.74it/s]

Epoch [20], Train Loss : [0.20758] Val Loss : [0.94006] Val F1 Score : [0.73444]


In [ ]:
torch.save({
    'model': model.state_dict(),
    'optim': optimizer.state_dict()
}, '20epoch_efficientnet_b4.tar')

In [ ]:
test_df = pd.read_csv('./test.csv')

In [ ]:
test_img_paths = get_data(test_df, infer=True)

In [ ]:
test_dataset = CustomDataset(test_img_paths, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False, num_workers=0)

In [ ]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    
    model_preds = []
    
    with torch.no_grad():
        for img in tqdm(iter(test_loader)):
            img = img.float().to(device)
            # h = size[0].to(device)
            # w = size[1].to(device)
            # h = h.view(h.size(0), 1)
            # w = h.view(w.size(0), 1)
            
            model_pred = model(img)
            model_preds += model_pred.argmax(1).detach().cpu().numpy().tolist()
    
    print('Done.')
    return model_preds

In [ ]:
preds = inference(infer_model, test_loader, args.device)

100%|██████████| 396/396 [02:00<00:00,  3.29it/s]

Done.


In [ ]:
preds = le.inverse_transform(preds) 

In [ ]:
submit = pd.read_csv('./sample_submission.csv')

In [ ]:
submit['artist'] = preds
submit.head()

,id,artist
0,TEST_00000,Edgar Degas
1,TEST_00001,Amedeo Modigliani
2,TEST_00002,Titian
3,TEST_00003,Albrecht Du rer
4,TEST_00004,Henri de Toulouse-Lautrec


In [ ]:
submit.to_csv('./efficient_b4_2mlp.csv', index=False)

In [ ]:
torch.cuda.memory_reserved()

1950351360

In [ ]:
torch.cuda.empty_cache()